# Saverlife Chatbot Demo
### Write a question and hit enter

In [55]:
# !pip install voila
# !jupyter serverextension enable --sys-prefix voila 

In [56]:
import ipywidgets as widgets
from fastai.text.all import *

In [57]:
# from platform import python_version
# print(python_version())
# import torch
# print(torch.__version__) 
# import fastai; fastai.__version__
# widgets.__version__
# import voila
# voila.__version__

In [59]:
def _end_headline(x):
    if type(x) != str: return ''
    if x[-1] in ['.', '?', '!', '…', ':', '“', '”']: return x + ' '
    else: return x + '. '

def prep_text(file = 'text-1-15c.csv'):
    df = pd.read_csv(file)
    df = df[~df.text.isna()].copy()
    df = df[['headline', 'segment', 'subheading', 'text',
                               'top_tag', 'url', 'text length']].copy()
    df['joincol'] = df['subheading']
    df.loc[df.joincol=='','joincol'] = df.loc[df.joincol=='','headline']
    df.joincol = df.joincol.map(_end_headline).values
    df = df[df.headline.map(lambda x: x[:33]) != "Ask an Expert episode 2 follow-up"]
    return(df)

def breakup_text(text, max=90):
    if len(text) < max:
        print(text)
    else:
        for i in range(max-1,1,-1):
            if text[i] == ' ':
                print(text[:i])
                break
        breakup_text(text[i:].strip())

# sdf = prep_text()
# sdf.to_pickle('segment_df.pkl')
sdf = pd.read_pickle('segment_df.pkl')

In [58]:
import urllib.request

MODEL_URL = "https://www.dropbox.com/s/tmu82r4pd9h0ek8/chat_model.pkl?dl=1"

urllib.request.urlretrieve(MODEL_URL, "model.pkl")

learn = load_learner("model.pkl")

In [68]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def chat_response(text, n):
    pred = learn.predict(text)
    df = pd.DataFrame(pred[2], columns = ['pct'])
    df['label_idx'] = df.index
    df = df.sort_values('pct', ascending=False).iloc[:50]
    url_df = pd.DataFrame(learn.dls.vocab[1])
    url_df['label_idx'] = url_df.index
    df = df.merge(url_df)
    df.columns = ['pct', 'idx', 'joincol']
    df = df.merge(sdf)
    df['score'] = df['pct'] / (df['text length'] + 50) * 10000
    df = df.sort_values('score', ascending=False)
    print(f'{color.RED}Here are our top {n} results:{color.END}')
    for i in range(0,n):
        print(color.BOLD + color.UNDERLINE)
        breakup_text(df.iloc[i]['headline'])
        print(color.END + color.BOLD + color.DARKCYAN)
        breakup_text(df.iloc[i]['subheading'])
        print(color.END)
        breakup_text(df.iloc[i]['text'])
        print(f"\n{color.GREEN}Read the full article on the SaverLife Blog")
        print(df.iloc[i]['url'])
        print(color.END)

In [71]:
textbox = widgets.Text(
    value='How do I save more?',
    placeholder='Write your quetion here!',
    description='Question:',
    disabled=False,
    continuous_update=False
)

slider = widgets.IntSlider(
    description='Responses', max=20, min=1, value=3, continuous_update=False)


out = widgets.interactive_output(chat_response, {'text': textbox, 'n': slider})

widgets.HBox([widgets.VBox([textbox, slider])])

In [66]:
# a.on_submit(chat_response)

In [72]:
widgets.HBox([out])

In [31]:
# a = widgets.Text(
#     value='How do I save more?',
#     placeholder='Paste ticket description here!',
#     description='Chat:',
#     disabled=False
# )
# def f(a):
#     print('{}'.format(a + '\n' + a))

# out = widgets.interactive_output(f, {'a': a})

# widgets.HBox([widgets.VBox([a]), out])